<a href="https://colab.research.google.com/github/JYL480/SkinCancerClassStreamlit/blob/main/SkinCancerPytorchClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### We will use the hugging face dataset by the training, dataloader will be done manually!

### Note that we will be using ViT base 16 for our model from pytorch!


In [ ]:
try:
  from datasets import load_dataset
except:
  print("Installing Dataset...")
  !pip install datasets
  from datasets import load_dataset

Installing Dataset...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstallin

In [ ]:
from datasets import load_dataset

data_dir = "data"
ds = load_dataset("marmal88/skin_cancer")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/9577 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2492 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1285 [00:00<?, ? examples/s]

In [ ]:
ds_train = ds["train"]
ds_test = ds["test"]
ds_validation = ds["validation"]

In [ ]:
ds_train_df = ds_train.to_pandas()
ds_train_df["dx"].value_counts()

,count
dx,
melanocytic_Nevi,6405
melanoma,1076
benign_keratosis-like_lesions,1048
basal_cell_carcinoma,487
actinic_keratoses,315
vascular_lesions,136
dermatofibroma,110


In [ ]:
ds_test.features

{'image': Image(mode=None, decode=True, id=None),
 'image_id': Value(dtype='string', id=None),
 'lesion_id': Value(dtype='string', id=None),
 'dx': Value(dtype='string', id=None),
 'dx_type': Value(dtype='string', id=None),
 'age': Value(dtype='float64', id=None),
 'sex': Value(dtype='string', id=None),
 'localization': Value(dtype='string', id=None)}

In [ ]:
import pandas as pd
df = pd.DataFrame(ds_test["dx"])
df.value_counts()

,count
0,
melanocytic_Nevi,858
melanoma,144
benign_keratosis-like_lesions,142
basal_cell_carcinoma,67
actinic_keratoses,42
vascular_lesions,18
dermatofibroma,14


In [ ]:
class_names = ['melanocytic_Nevi', "melanoma", "benign_keratosis-like_lesions",
                    'basal_cell_carcinoma', 'actinic_keratoses',"vascular_lesions",
                    'dermatofibroma']

In [ ]:
class_to_index = {name:index for index, name in enumerate (class_names)}
class_to_index

{'melanocytic_Nevi': 0,
 'melanoma': 1,
 'benign_keratosis-like_lesions': 2,
 'basal_cell_carcinoma': 3,
 'actinic_keratoses': 4,
 'vascular_lesions': 5,
 'dermatofibroma': 6}

In [ ]:
# What next? Get the transformation of the image!!
# You can get it automatically from pytorch ViT

# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

In [ ]:
# Get the weights and auto transform of the ViT model

weights = torchvision.models.ViT_B_16_Weights.DEFAULT
auto_transform = weights.transforms()
auto_transform

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

- So images that are passed in will be transformed into that dimension!!

In [ ]:
ds_test.set_format(type='torch', columns=['image', 'dx'])

### LOL you can use this shortcut to preprocess this data!! Into a dictionary of the things you want!

In [ ]:
ds_train.set_format(type='torch', columns=['image', 'dx'])

In [ ]:
ds_train[0]

{'image': tensor([[[167, 167, 167,  ..., 193, 196, 197],
          [167, 167, 168,  ..., 191, 194, 196],
          [166, 168, 168,  ..., 189, 190, 193],
          ...,
          [151, 151, 152,  ..., 162, 162, 162],
          [151, 151, 151,  ..., 164, 162, 161],
          [152, 150, 150,  ..., 164, 161, 161]],
 
         [[139, 139, 139,  ..., 159, 159, 163],
          [139, 139, 138,  ..., 157, 158, 162],
          [138, 140, 140,  ..., 155, 156, 159],
          ...,
          [120, 120, 121,  ..., 131, 131, 131],
          [120, 120, 120,  ..., 130, 128, 130],
          [121, 119, 119,  ..., 130, 127, 130]],
 
         [[128, 128, 127,  ..., 149, 150, 153],
          [127, 127, 128,  ..., 147, 146, 152],
          [126, 128, 129,  ..., 146, 146, 149],
          ...,
          [102, 100, 101,  ..., 128, 128, 128],
          [100, 100, 100,  ..., 129, 127, 128],
          [101,  99,  99,  ..., 128, 125, 127]]], dtype=torch.uint8),
 'dx': 'actinic_keratoses'}

In [ ]:
# We will create a custom dataset bah

from torch.utils.data import Dataset

class SkinClassificationDataset(Dataset):
  def __init__(self, data,transform = None):
    self.data = data
    self.transform = transform
    self.classes = class_names
    self.class_to_idx = class_to_index

  def load_image(self, index):
    image = self.data[index]["image"]
    return image

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    # We need to return img and label, so your str lable will have to become index!!
    img = self.load_image(index)
    label = self.data[index]["dx"]
    label = self.class_to_idx[label]
    if self.transform:
      return self.transform(img), label
    else:
      return img, label


- We will now try and test it out!

In [ ]:
train_data_custom = SkinClassificationDataset(data = ds_train, transform = auto_transform)
test_data_custom = SkinClassificationDataset(data = ds_test, transform = auto_transform)

In [ ]:
train_data_custom.__len__()

9577

### Yay our custom dataset seems like it is working LOL

In [ ]:
test_data_custom.__len__()

1285

### Place the images into a dataloader from pytorch!

In [ ]:
import os
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset = train_data_custom, batch_size = 64, num_workers=os.cpu_count(), shuffle = True)
test_dataloader = DataLoader(dataset = test_data_custom, batch_size = 64, num_workers=os.cpu_count(),  shuffle = False)


In [ ]:
# I will test the dataloader

img_custom, label_custom = next(iter(train_dataloader))
print(f"Image shape: {img_custom.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label_custom.shape}") #Batch of 64 labels loh!


Image shape: torch.Size([64, 3, 224, 224]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([64])


### To initalise the model and then train it!

In [ ]:
#Agnostic code!
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
weights = torchvision.models.ViT_B_16_Weights.DEFAULT
model = torchvision.models.vit_b_16(weights = weights).to(device)

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:03<00:00, 91.8MB/s]


In [ ]:
try:
  import torchinfo #If there is one aldy
except:
  !pip install torchinfo
  import torchinfo

# Install this ot have more info about what is in teh model!

In [ ]:
from torchinfo import summary

# Print a summary using torchinfo (uncomment for actual output)
summary(model=model,
        input_size=(64, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [64, 3, 224, 224]    [64, 1000]           768                  True
├─Conv2d (conv_proj)                                         [64, 3, 224, 224]    [64, 768, 14, 14]    590,592              True
├─Encoder (encoder)                                          [64, 197, 768]       [64, 197, 768]       151,296              True
│    └─Dropout (dropout)                                     [64, 197, 768]       [64, 197, 768]       --                   --
│    └─Sequential (layers)                                   [64, 197, 768]       [64, 197, 768]       --                   True
│    │    └─EncoderBlock (encoder_layer_0)                   [64, 197, 768]       [64, 197, 768]       7,087,872            True
│    │    └─EncoderBlock (encoder_layer_1)                   [64, 197, 768]       [64, 197, 76

#### Notice that it has 1000 output we will have to change that to 7 only!
- We will do this by freezing the model to change the output (depends)
- Maybe we dont have to freeze if we want to fine tune better

In [ ]:
for param in model.encoder.parameters():
    param.requires_grad = False

for param in model.conv_proj.parameters():
    param.requires_grad = False

In [ ]:
summary(model=model,
        input_size=(64, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [64, 3, 224, 224]    [64, 1000]           768                  Partial
├─Conv2d (conv_proj)                                         [64, 3, 224, 224]    [64, 768, 14, 14]    (590,592)            False
├─Encoder (encoder)                                          [64, 197, 768]       [64, 197, 768]       151,296              False
│    └─Dropout (dropout)                                     [64, 197, 768]       [64, 197, 768]       --                   --
│    └─Sequential (layers)                                   [64, 197, 768]       [64, 197, 768]       --                   False
│    │    └─EncoderBlock (encoder_layer_0)                   [64, 197, 768]       [64, 197, 768]       (7,087,872)          False
│    │    └─EncoderBlock (encoder_layer_1)                   [64, 197, 768]       [64, 

In [ ]:
output_shape = len(class_names)
output_shape

7

In [ ]:
# Now we can eddit the classifier output_features to only 3 = pizza, steak and sushi!!
# Use oop where you haev model.classifier  = then rewrite the whole thing!
# Recreate the classifier layer and seed it to the target device
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Get the length of class_names (one output unit for each class)
output_shape = len(class_names)

# Recreate the classifier layer and seed it to the target device
model.heads = torch.nn.Sequential(
    torch.nn.Linear(in_features=768,
                    out_features=output_shape, # same number of output units as our number of classes
                    bias=True)).to(device)

In [ ]:
summary(model=model,
        input_size=(64, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [64, 3, 224, 224]    [64, 7]              768                  Partial
├─Conv2d (conv_proj)                                         [64, 3, 224, 224]    [64, 768, 14, 14]    (590,592)            False
├─Encoder (encoder)                                          [64, 197, 768]       [64, 197, 768]       151,296              False
│    └─Dropout (dropout)                                     [64, 197, 768]       [64, 197, 768]       --                   --
│    └─Sequential (layers)                                   [64, 197, 768]       [64, 197, 768]       --                   False
│    │    └─EncoderBlock (encoder_layer_0)                   [64, 197, 768]       [64, 197, 768]       (7,087,872)          False
│    │    └─EncoderBlock (encoder_layer_1)                   [64, 197, 768]       [64, 

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Using these 2 lol


In [ ]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

In [ ]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
      # Loop through DataLoader batches
    for batch, (X, y) in enumerate(dataloader):
    # Send data to GPU
    # PLace the data on the same device!!!!!
      X, y = X.to(device), y.to(device)
      # Note the X is the image and y is the label!
      # do the forward pass
      # Pass in the image to the model
      test_pred = model(X)
      # Note this output is logits as we are doing classification.
      # So you might have to put through a function or use argmax!

      # Calculate the loss and accuracy
      # 1st = input, 2nd = target (the correct one)
      # Because crossentropyloss input = logits so we can use test_pred
      # the raw logits will be put through a softmax function which will give the prob
      # The true index of hte class will be compared with the probabilty hence giving the loss
      loss = loss_fn(test_pred, y)
      test_loss += loss

      # This is for the accuracy
      # We can just use the argmax on the raw logits to get the class index predicted
      test_pred_labels = test_pred.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
      # Adjust metrics to get average loss and accuracy per batch

    # You will get the average!!
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc



In [ ]:
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn,
          epochs):

    # 2. Create empty results dictionary
    # To track results in each epoch!
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)

        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # 6. Return the filled results at the end of the epochs
    return results

In [ ]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Setup training and save the results
results = train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs= 20)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.8002 | train_acc: 0.7198 | test_loss: 0.6175 | test_acc: 0.7801
Epoch: 2 | train_loss: 0.5936 | train_acc: 0.7831 | test_loss: 0.5184 | test_acc: 0.8177
Epoch: 3 | train_loss: 0.5371 | train_acc: 0.8037 | test_loss: 0.4788 | test_acc: 0.8326
Epoch: 4 | train_loss: 0.4998 | train_acc: 0.8184 | test_loss: 0.4458 | test_acc: 0.8430
Epoch: 5 | train_loss: 0.4717 | train_acc: 0.8273 | test_loss: 0.4233 | test_acc: 0.8497
Epoch: 6 | train_loss: 0.4527 | train_acc: 0.8316 | test_loss: 0.4155 | test_acc: 0.8616
Epoch: 7 | train_loss: 0.4354 | train_acc: 0.8387 | test_loss: 0.3915 | test_acc: 0.8638
Epoch: 8 | train_loss: 0.4192 | train_acc: 0.8457 | test_loss: 0.3848 | test_acc: 0.8638
Epoch: 9 | train_loss: 0.4081 | train_acc: 0.8511 | test_loss: 0.3776 | test_acc: 0.8683
Epoch: 10 | train_loss: 0.3991 | train_acc: 0.8549 | test_loss: 0.3726 | test_acc: 0.8638
Epoch: 11 | train_loss: 0.3914 | train_acc: 0.8587 | test_loss: 0.3672 | test_acc: 0.8757
Epoch: 12 | train_l

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_curves(results: dict[str, list[float]]):
    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_acc": [...],
             "test_loss": [...],
             "test_acc": [...]}
    """

    # Ensure the loss and accuracy values are on the CPU and converted to lists
    loss = [val.cpu().item() if isinstance(val, torch.Tensor) else val for val in results['train_loss']]
    test_loss = [val.cpu().item() if isinstance(val, torch.Tensor) else val for val in results['test_loss']]
    accuracy = [val.cpu().item() if isinstance(val, torch.Tensor) else val for val in results['train_acc']]
    test_accuracy = [val.cpu().item() if isinstance(val, torch.Tensor) else val for val in results['test_acc']]

    # Figure out how many epochs there were
    epochs = range(len(loss))

    # Setup a plot
    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, test_loss, label='test_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='train_accuracy')
    plt.plot(epochs, test_accuracy, label='test_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()


plot_loss_curves(results)

NameError: name 'results' is not defined

In [ ]:
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.

    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

    Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)

In [ ]:
# I will save the model into a pickle file!!
# In the future i will load this embeddings into the model when i want to deploy!

In [ ]:

save_model(model=model,
                 target_dir="models",
                 model_name="SkinCancerClassification.pth")

[INFO] Saving model to: models/SkinCancerClassification.pth


In [ ]:
from typing import Tuple, Dict
from time import perf_counter as timer
model.to(device)

def predict(image) -> Tuple[Dict, float]:
    start_time = timer()
    # Transform image
    image = auto_transform(image).unsqueeze(0).to(device) #Need to remove the first dim
    model.eval()
    with torch.inference_mode():
        # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
        pred_probs = torch.softmax(model(image), dim=1)

    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}


    pred_time = round(timer() - start_time, 5)
    return pred_labels_and_probs, pred_time


In [ ]:
from PIL import Image
image_path = "actinic_keratoses2.jpg"
image = Image.open(image_path)


pred_labels_and_probs, pred_time = predict(image)

print(f"Predictions: {pred_labels_and_probs}")
print(f"Prediction time: {pred_time} seconds")

Predictions: {'melanocytic_Nevi': 0.08426766097545624, 'melanoma': 0.3281534016132355, 'benign_keratosis-like_lesions': 0.3311505913734436, 'basal_cell_carcinoma': 0.002748921513557434, 'actinic_keratoses': 0.2532218098640442, 'vascular_lesions': 2.3761833290336654e-05, 'dermatofibroma': 0.0004337789141573012}
Prediction time: 0.03266 seconds


from google.colab import files

files.download("models/SkinCancerClassification.pth")